In [62]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
import os   
import cv2
import tqdm
import zigzag_helper
from    tensorflow import keras
from    tensorflow.keras import layers,Sequential,optimizers,losses,metrics
from    tqdm import tqdm
from    sklearn import preprocessing
from    sklearn.preprocessing import OneHotEncoder
from    pandas import DataFrame

print('tensorflow version:',tf.__version__)
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ["CUDA_VISIBLE_DEVICES"]="0"
pd.set_option('max_columns',200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
tensorflow version: 2.6.0


In [63]:
def Zigzag(arrary,size):
    #锯齿形编码填充
    zigzag=np.zeros([size,size])
    row=zigzag.shape[0]
    colum=zigzag.shape[1]
    i,j,k=0,0,0
    while i < row and j < colum and k < len(arrary):
        zigzag[i,j]=arrary[k]
        k+=1
        #i+j为偶数，向右上方向移动
        if (i+j)%2==0 :
            #如果右边界超出，则向下
            if (i-1) in range(row) and (j+1) not in range(colum):
                i+=1
            #如果上边界超出，则向下
            elif (i-1) not in range(row) and (j+1) in range(colum):
                j+=1
            #如果右上边界都超出，则向下
            elif (i-1) not in range(row) and (j+1) not in range(colum):
                i+=1
            else:
                i-=1
                j+=1
        #i+j为奇数，则向左下移动
        elif (i+j)%2==1:
            #如果左边界超出，则向下
            if (i+1) in range(row) and (j-1) not in range(colum):
                i+=1
            #如果上边界超出，则向下
            elif (i+1) not in range(row) and (j-1) in range(colum):
                j+=1
            #如果右上边界都超出，则向下
            elif (i+1) not in range(row) and (j-1) not in range(colum):
                j+=1
            else:
                i+=1
                j-=1
    return zigzag

In [64]:
def Load_img():
    train_file_dir='KDDCUP99_TRAIN_IMGSET(16)'
    ds_train=keras.preprocessing.image_dataset_from_directory(directory=train_file_dir,
                                                    labels='inferred',
                                                    label_mode='categorical',
                                                    class_names=None,
                                                    color_mode='grayscale',
                                                    shuffle=True,
                                                    batch_size=256,
                                                    image_size=(16,16))
    ds_train=ds_train.map(lambda x,y:(x*1/255.,y))
    return ds_train

In [65]:
def muil_to_bin(x):
    if x>0:
        return 1
    else:
        return 0

In [66]:
def load_train():
    print('开始载入数据')
    handled_data=pd.read_csv('KDDCUP99_PREPROCESSED_WITHLABELS.csv',header=None)
    labels=pd.read_csv('KDDCUP99_AfterNumerical.csv',header=None)
    
    y=labels.iloc[:,-1]
    y=y.map(muil_to_bin)
    
    temp=DataFrame(handled_data.iloc[:,0:-1])
    x=[]
    img_size=16
    with tqdm(total=temp.shape[0]) as t:
        for i in range(temp.shape[0]):
            img=Zigzag(np.array(temp.iloc[i]),img_size)
            img_idct=cv2.idct(img.flatten())
            img_idct=img_idct.reshape([img_size,img_size])
            img_idct=tf.reshape(img_idct,[16,16,1])
            x.append(img_idct)
            t.update(1)

    x_train=tf.convert_to_tensor(x,dtype=tf.float32)/255.
    y_train=tf.convert_to_tensor(y,dtype=tf.int32)
    
    y_onehot=tf.one_hot(y_train,depth=2)

    ds_train=tf.data.Dataset.from_tensor_slices((x_train,y_onehot)).shuffle(8748454).batch(256)
    
    print('数据载入，生成datset完成')
    return ds_train

In [67]:
def load_val():
    input_file='./kddcup99/kddcup.data.corrected'
    out_file='KDDCUP99_VALDATA.csv'
    zigzag_helper.preprocess(in_file=input_file,out_file=out_file,ifsample=1,sample_num=0,sample_frac=0.02)
    val_data=pd.read_csv(out_file,header=None)
    x=val_data.iloc[:,0:-1]
    y=val_data.iloc[:,-1]

    x_val=[]

    for i in range(x.shape[0]):
        temp=np.array(x.iloc[i])
        img=zigzag_helper.zizagcode(temp,16)
        img=tf.reshape(img,[16,16,1])
        x_val.append(img)
        
    x_val=tf.convert_to_tensor(x_val,dtype=tf.float32)/255.
    y_val=tf.convert_to_tensor(y,dtype=tf.int32)
    y_val=tf.one_hot(y_val,depth=2)

    ds_val=tf.data.Dataset.from_tensor_slices((x_val,y_val)).shuffle(13124).batch(256)
    
    return ds_val



In [68]:
#搭建网络
#input=[256,16,16,1]
network=Sequential([
    layers.Conv2D(16,kernel_size=[3,3],strides=1,padding='same',activation='relu'),
    layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'),
    
    layers.Conv2D(32,kernel_size=[3,3],strides=1,padding='same',activation='relu'),
    layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'),
    
    layers.Flatten(),
    layers.Dense(256,activation='sigmoid'),
    layers.Dropout(0.4),
    layers.Dense(128,activation='sigmoid'),
    layers.Dropout(0.4),
    layers.Dense(2,activation='softmax')
])
network.build(input_shape=[None,16,16,1])
network.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 16, 16, 16)        160       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 32)          4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)              

In [69]:
def train(ds_train,ds_val):
    print('开始训练')
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)

    tf_callbcak=tf.keras.callbacks.TensorBoard(log_dir=train_log_dir)
    
    METRICS=[
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall')
    ]
    
    network.compile(optimizer=optimizers.Adam(learning_rate=6e-4,
                                              ),
                    loss='binary_crossentropy',
                    metrics=METRICS,
                    )
    network.fit(ds_train,
                validation_data=ds_val,
                epochs=200,
                callbacks=tf_callbcak)
    

In [70]:
ds_train=load_train()

开始载入数据


100%|██████████| 494021/494021 [06:35<00:00, 1248.57it/s]


数据载入，生成datset完成


In [71]:
ds_val=load_val()

d:\Anaconda\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
d:\Anaconda\envs\tensorflow\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
d:\Anaconda\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
d:\Anaconda\envs\tensorflow\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
d:\Anaconda\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` 

afr_oh: (97969, 126)
(97969, 123)


In [72]:
train(ds_train,ds_val)
network.save('model_zigzag.h5')

开始训练
Epoch 1/200
1930/1930 [==============================] - 292s 149ms/step - loss: 0.0527 - tp: 485954.0000 - fp: 8067.0000 - tn: 485954.0000 - fn: 8067.0000 - accuracy: 0.9837 - precision: 0.9837 - recall: 0.9837 - val_loss: 0.0077 - val_tp: 97814.0000 - val_fp: 155.0000 - val_tn: 97814.0000 - val_fn: 155.0000 - val_accuracy: 0.9984 - val_precision: 0.9984 - val_recall: 0.9984
Epoch 2/200
1930/1930 [==============================] - 269s 139ms/step - loss: 0.0158 - tp: 491627.0000 - fp: 2394.0000 - tn: 491627.0000 - fn: 2394.0000 - accuracy: 0.9952 - precision: 0.9952 - recall: 0.9952 - val_loss: 0.0035 - val_tp: 97868.0000 - val_fp: 101.0000 - val_tn: 97868.0000 - val_fn: 101.0000 - val_accuracy: 0.9990 - val_precision: 0.9990 - val_recall: 0.9990
Epoch 3/200
1930/1930 [==============================] - 283s 146ms/step - loss: 0.0079 - tp: 492817.0000 - fp: 1204.0000 - tn: 492817.0000 - fn: 1204.0000 - accuracy: 0.9976 - precision: 0.9976 - recall: 0.9976 - val_loss: 0.0052 - val_